# This notebook generates the data defining the case study

A .csv file is created containing:
 - Origin (latitude; longitude; altitude)
 - Destination (latitude; longitude; altitude)
 - Time of departure (formatted; only in seconds)
 - Drone model
 - Purpose
 - Time of planning

For each drone forming the background traffic.

In [ ]:
from pathlib import Path

"""
We define a default path where the modules developed in USEPE are stored
"""

bluesky_path = Path(Path.home(), 'ws', 'bluesky')

"""
We define a second default path where the scenarios are stored
"""

scenarios_path = Path(Path.home(), 'ws', 'scenarios', 'exercise_4')

In [ ]:
import os
import sys
import configparser
import pickle
import osmnx as ox
import random
import pandas as pd
import datetime

os.chdir(bluesky_path)
from usepe.city_model.utils import read_my_graphml, layersDict
from usepe.city_model.multi_di_graph_3D import MultiDiGrpah3D
from usepe.city_model.city_graph import cityGraph
from usepe.city_model.scenario_definition import createSurveillanceCSV, createATMcsv, createBackgroundTrafficCSV, createScenarioCSV


[City]

mode = rectangle

hannover_lat = 

hannover_lon = 

zone_size = 1000 

hannover_lat_min = 52.373136

hannover_lat_max = 52.427466

hannover_lon_min = 9.771070

hannover_lon_max = 9.859913

import = True

imported_graph_path = data/exe_corridor_focus_ref.graphml


[Layers]

number_of_layers = 9

layer_width = 25


[Corridors]

corridors = 

altitude = 250

delta_z = 25

speed = 40

acceleration_length = 50

file_path_corridors = data/usepe-hannover-corridors.geojson


[Segments]

import = True

imported_segments_file_path = data/Hannover_initial_segments_ref.json


[BuildingData]

lat_min = 52.373136

lat_max = 52.427466

lon_min = 9.771070

lon_max = 9.859913

divisions = 20

directory_hannover = /home/ror/ws/bluesky/usepe/city_model/data/hannover_building_data


[Options]

one_way = False

simplify = True

simplification_distance = 0

simplification_error = 2


[Outputs]

graph_path = data/exe_corridor_focus_ref.graphml


[Strategic_Deconfliction]

ratio = 3

delay = 60


In [ ]:
 # -------------- 1. CONFIGURATION FILE -----------------
"""
This section reads the configuration file.
Change the config_path to read the desired file
"""
# CONFIG
config_path = Path(scenarios_path, 'settings_exe_4_ref.cfg')
config = configparser.ConfigParser()
config.read(config_path)

In [ ]:
# -------------- 2. CITY GRAPH -------------------------
"""
This section creates a city graph or loads the graph defined in the 'City' section of the
configuration file.
"""
# City
os.chdir(scenarios_path)
if config['City'].getboolean('import'):
    filepath = config['City']['imported_graph_path']
    G = read_my_graphml(filepath)
    G = MultiDiGrpah3D(G)
else:
    print('Check the path to the graph file')
    G = cityGraph(config)

In [ ]:
# -------------- 3. SEGMENTS ----------------------------
"""
This section creates an airspace segmentation or loads the segmentation defined in the 'Segments'
section of the configuration file.
Comment it to neglect the segmentation
"""
os.chdir(scenarios_path)
if config['Segments'].getboolean('import'):
    segments = pd.read_json(config['Segments']['imported_segments_file_path'], orient="records", lines=True)
else:
    segments = None

# Data for the experiments

The densities to be tested in the experiments are
 - Medium: 0.964 simultaneous flights / km^2
 
The average flight duration calculated is: 283 s

The simulation time considered is: 30 min (1800 s)


In [ ]:
# General
simulation_time = 1800

# Background Traffic

In [ ]:
# Background traffic
density = {'Low': 0.0964, 'Medium': 0.964, 'High': 9.64}

background_traffic = {'density': density['Medium'],
                      'avg_flight_duration': 283,
                      'simulation_time': simulation_time,
                      'G': G,
                      'segments': segments,
                      'config': config,
                      'default_path': bluesky_path}


# Combined flight plans

In [ ]:
file_name = 'EXE-4-Medium'

nPlans = 10

createScenarioCSV(file_name, nPlans, background_traffic=background_traffic)